### Evaluation of GAMD-large trained on RBPE data

In [5]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from nn_module import WaterMDDynamicBoxNet
from train_utils import WaterDataRealLarge
from train_network_real_large import ParticleNetLightning
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataRealLarge(dataset_path='./md_dataset/RPBE-data-processed.npz', mode='test')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                              'box_size': [batch['box_size'] for batch in batches],
                          })

PATH = './model_ckpt_reallarge_large/checkpoint_950.ckpt'
SCALER_CKPT = './model_ckpt_reallarge_large/scaler_950.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=512,
                       hidden_dim=512,
                       edge_embedding_dim=768,
                      drop_edge=False,
                       conv_layer=5,
                      cutoff=9.5,
                      rotate_aug=False,
                       update_edge=False,
                       use_part=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')

Using 6518 training samples
Using 723 testing samples
(7241,)
Using following set of hyper-parameters
namespace(conv_layer=5, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=768, encoding_size=512, hidden_dim=512, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)
Using following set of hyper-parameters
namespace(conv_layer=5, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=768, encoding_size=512, hidden_dim=512, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)



In [6]:
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos, box_size = batch['feat'].float().cuda(), batch['pos'], batch['box_size']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0], box_size[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(27/0.52917)  # hatree / bohr to ev/A
        gt = gt.view(-1, 3)*(27/0.52917)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1

Finished 0
Finished 200
Finished 400
Finished 600


In [7]:
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Ralative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Ralative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

cos value: 0.9928294846985093
MAE 0.019174728339468112
std of MAE per sample: 0.014027951839411648
RMSE: 0.028895289064118902
std of RMSE per sample: 0.02047078281397487
Ralative MAE: 0.011596697536030594
Ralative std: 0.00848397493052428


### Evaluation of GAMD-small trained on RBPE data

In [2]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from nn_module import WaterMDDynamicBoxNet
from train_utils import WaterDataRealLarge
from train_network_real_large import ParticleNetLightning
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataRealLarge(dataset_path='./md_dataset/RPBE-data-processed.npz', mode='test')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                              'box_size': [batch['box_size'] for batch in batches],
                          })

PATH = './model_ckpt_reallarge_small/checkpoint_950.ckpt'
SCALER_CKPT = './model_ckpt_reallarge_small/scaler_950.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=128,
                       hidden_dim=128,
                       edge_embedding_dim=128,
                      drop_edge=False,
                       conv_layer=4,
                      cutoff=9.5,
                      rotate_aug=False,
                       update_edge=False,
                       use_part=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos, box_size = batch['feat'].float().cuda(), batch['pos'], batch['box_size']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0], box_size[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(27/0.52917)  # hatree / bohr to ev/A
        gt = gt.view(-1, 3)*(27/0.52917)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1
        
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Relative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Relative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

Using 6518 training samples
Using 723 testing samples
(7241,)
Using following set of hyper-parameters
namespace(conv_layer=4, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=128, encoding_size=128, hidden_dim=128, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)
Using following set of hyper-parameters
namespace(conv_layer=4, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=128, encoding_size=128, hidden_dim=128, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)

Finished 0
Finished 200
Finished 400
Finished 600
cos value: 0.9795223481269464
MAE 0.033374673326953125
std of MAE per sample: 0.02098847054314041
RMSE: 0.04776421624519811
std of RMSE per sample: 0.029565195651719958
Relative MAE: 0.020184692324420137
Relative std: 0.012693631968265928


### Evaluation of GAMD using only part of the data

In [4]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from nn_module import WaterMDDynamicBoxNet
from train_utils import WaterDataRealLarge
from train_network_real_large import ParticleNetLightning
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataRealLarge(dataset_path='./md_dataset/RPBE-data-processed.npz', mode='test')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                              'box_size': [batch['box_size'] for batch in batches],
                          })

PATH = './model_ckpt_reallarge_part/checkpoint_950.ckpt'
SCALER_CKPT = './model_ckpt_reallarge_part/scaler_950.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=256,
                       hidden_dim=128,
                       edge_embedding_dim=256,
                      drop_edge=False,
                       conv_layer=5,
                      cutoff=9.5,
                      rotate_aug=False,
                       update_edge=False,
                       use_part=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos, box_size = batch['feat'].float().cuda(), batch['pos'], batch['box_size']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0], box_size[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(27/0.52917)  # hatree / bohr to ev/A
        gt = gt.view(-1, 3)*(27/0.52917)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1
        
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Relative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Relative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

Using 6518 training samples
Using 723 testing samples
(7241,)
Using following set of hyper-parameters
namespace(conv_layer=5, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=256, encoding_size=256, hidden_dim=128, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)
Using following set of hyper-parameters
namespace(conv_layer=5, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=256, encoding_size=256, hidden_dim=128, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)

Finished 0
Finished 200
Finished 400
Finished 600
cos value: 0.9621473527852384
MAE 0.0761859785255583
std of MAE per sample: 0.03553132046224863
RMSE: 0.10641007862431771
std of RMSE per sample: 0.04969007950127709
Relative MAE: 0.04607657192351802
Relative std: 0.021489012473169773


### GAMD on TIP3P

In [ ]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from train_utils import WaterDataNew
from train_network_tip3p import ParticleNetLightning, NUM_OF_ATOMS
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataNew(dataset_path=os.path.join('./md_dataset', 'water_data_tip3p'),
                               sample_num=1000,
                               case_prefix='data_',
                               seed_num=10,
                               m_num=NUM_OF_ATOMS//3,
                               mode='test',
                               data_type='tip3p')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                          })

PATH = './model_ckpt_tip3pnew_long/checkpoint_28.ckpt'
SCALER_CKPT = './model_ckpt_tip3pnew_long/scaler_28.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=128,
                       hidden_dim=128,
                       edge_embedding_dim=128,
                      drop_edge=False,
                      rotate_aug=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')

In [3]:
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos = batch['feat'].float().cuda(), batch['pos']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(0.0010364)  # KJ/mol to ev/A
        gt = gt.view(-1, 3)*(0.0010364)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1

Finished 0
Finished 200
Finished 400
Finished 600
Finished 800


In [4]:
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Ralative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Ralative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

cos value: 0.9996882728811234
MAE 0.011263003440500439
std of MAE per sample: 0.0008416153970639325
RMSE: 0.015159353023402752
std of RMSE per sample: 0.0011593059574419755
Ralative MAE: 0.011608358915583617
Ralative std: 0.000867421700580201


### GAMD on TIP4P

In [2]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from train_utils import WaterDataNew
from train_network_tip4p import ParticleNetLightning, NUM_OF_ATOMS
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataNew(dataset_path=os.path.join('./md_dataset', 'water_data_tip4p'),
                               sample_num=1000,
                               case_prefix='data_',
                               seed_num=10,
                               m_num=NUM_OF_ATOMS//3,
                               mode='test',
                               data_type='tip4p')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                          })

PATH = './model_ckpt_tip4pnew3_marked/checkpoint_19.ckpt'
SCALER_CKPT = './model_ckpt_tip4pnew3_marked/scaler_19.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=128,
                       hidden_dim=128,
                       edge_embedding_dim=128,
                      drop_edge=False,
                      rotate_aug=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')

Using backend: pytorch


Including atom type: 2
Using following set of hyper-parameters
{'in_feats': 1, 'encoding_size': 128, 'out_feats': 3, 'bond': array([[  0,   1],
       [  0,   2],
       [  3,   4],
       ...,
       [747, 749],
       [750, 751],
       [750, 752]]), 'hidden_dim': 128, 'edge_embedding_dim': 128, 'conv_layer': 4, 'drop_edge': False, 'use_layer_norm': True, 'box_size': 20.0}
Using following set of hyper-parameters
{'in_feats': 1, 'encoding_size': 128, 'out_feats': 3, 'bond': array([[  0,   1],
       [  0,   2],
       [  3,   4],
       ...,
       [747, 749],
       [750, 751],
       [750, 752]]), 'hidden_dim': 128, 'edge_embedding_dim': 128, 'conv_layer': 4, 'drop_edge': False, 'use_layer_norm': True, 'box_size': 20.0}



In [3]:
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
o_ratios = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos = batch['feat'].float().cuda(), batch['pos']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(0.0010364)  # KJ/mol to ev/A
        gt = gt.view(-1, 3)*(0.0010364)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1

Finished 0
Finished 200
Finished 400
Finished 600
Finished 800


In [ ]:
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Ralative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Ralative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

### GAMD-med on RPBE data

In [10]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from nn_module import WaterMDDynamicBoxNet
from train_utils import WaterDataRealLarge
from train_network_real_large import ParticleNetLightning
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataRealLarge(dataset_path='./md_dataset/RPBE-data-processed.npz', mode='test')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                              'box_size': [batch['box_size'] for batch in batches],
                          })

PATH = './model_ckpt_reallarge_reg/checkpoint_798.ckpt'
SCALER_CKPT = './model_ckpt_reallarge_reg/scaler_798.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=256,
                       hidden_dim=128,
                       edge_embedding_dim=256,
                      drop_edge=False,
                       conv_layer=5,
                      cutoff=9.5,
                      rotate_aug=False,
                       update_edge=False,
                       use_part=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos, box_size = batch['feat'].float().cuda(), batch['pos'], batch['box_size']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0], box_size[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(27/0.52917)  # hatree / bohr to ev/A
        gt = gt.view(-1, 3)*(27/0.52917)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1
        
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Relative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Relative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

Using 6518 training samples
Using 723 testing samples
(7241,)
Using following set of hyper-parameters
namespace(conv_layer=5, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=256, encoding_size=256, hidden_dim=128, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)
Using following set of hyper-parameters
namespace(conv_layer=5, cutoff=9.5, data_dir='', drop_edge=False, edge_embedding_dim=256, encoding_size=256, hidden_dim=128, loss='mae', rotate_aug=False, update_edge=False, use_layer_norm=True, use_part=False)

Finished 0
Finished 200
Finished 400
Finished 600
cos value: 0.9858087938310441
MAE 0.02428514585951099
std of MAE per sample: 0.016796812913669183
RMSE: 0.03538629214569198
std of RMSE per sample: 0.02309355171008023
Relative MAE: 0.014687430568257363
Relative std: 0.01015855638111833


In [3]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from nn_module import WaterMDDynamicBoxNet
from train_utils import WaterDataNew
from train_network_tip3p import ParticleNetLightning, NUM_OF_ATOMS
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataNew(dataset_path=os.path.join('./md_dataset', 'water_data_tip3p'),
                               sample_num=1000,
                               case_prefix='data_',
                               seed_num=10,
                               m_num=NUM_OF_ATOMS//3,
                               mode='test',
                               data_type='tip3p')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                          })

PATH = './model_ckpt_no_expand/checkpoint_20.ckpt'
SCALER_CKPT = './model_ckpt_no_expand/scaler_20.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=128,
                       hidden_dim=128,
                       edge_embedding_dim=128,
                      drop_edge=False,
                      rotate_aug=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos = batch['feat'].float().cuda(), batch['pos']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(0.0010364)  # KJ/mol to ev/A
        gt = gt.view(-1, 3)*(0.0010364)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1
        
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Relative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Relative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

Including atom type: 2
Using following set of hyper-parameters
{'in_feats': 1, 'encoding_size': 128, 'out_feats': 3, 'hidden_dim': 128, 'edge_embedding_dim': 128, 'conv_layer': 4, 'drop_edge': False, 'use_layer_norm': True, 'box_size': 20.0}
Using following set of hyper-parameters
{'in_feats': 1, 'encoding_size': 128, 'out_feats': 3, 'hidden_dim': 128, 'edge_embedding_dim': 128, 'conv_layer': 4, 'drop_edge': False, 'use_layer_norm': True, 'box_size': 20.0}

Finished 0
Finished 200
Finished 400
Finished 600
Finished 800
cos value: 0.9996864148707305
MAE 0.01103301183405695
std of MAE per sample: 0.0008425929414812384
RMSE: 0.014894169469857357
std of RMSE per sample: 0.0011784660567412755
Relative MAE: 0.011371315117339954
Relative std: 0.0008684292192684407


In [5]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from train_utils import WaterDataNew
from train_network_tip4p import ParticleNetLightning, NUM_OF_ATOMS
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataNew(dataset_path=os.path.join('./md_dataset', 'water_data_new'),
                               sample_num=1000,
                               case_prefix='data_',
                               seed_num=10,
                               m_num=NUM_OF_ATOMS//3,
                               mode='test',
                               data_type='tip4p')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                          })

PATH = './model_ckpt_tip4p_nobond/checkpoint_10.ckpt'
SCALER_CKPT = './model_ckpt_tip4p_nobond/scaler_10.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=128,
                       hidden_dim=128,
                       edge_embedding_dim=128,
                      drop_edge=False,
                      rotate_aug=False,
                      data_dir='',
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos = batch['feat'].float().cuda(), batch['pos']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(0.0010364)  # KJ/mol to ev/A
        gt = gt.view(-1, 3)*(0.0010364)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1
        
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Relative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Relative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')

Including atom type: 2
Using following set of hyper-parameters
{'in_feats': 1, 'encoding_size': 128, 'out_feats': 3, 'hidden_dim': 128, 'edge_embedding_dim': 128, 'conv_layer': 4, 'drop_edge': False, 'use_layer_norm': True, 'box_size': 20.0}
Using following set of hyper-parameters
{'in_feats': 1, 'encoding_size': 128, 'out_feats': 3, 'hidden_dim': 128, 'edge_embedding_dim': 128, 'conv_layer': 4, 'drop_edge': False, 'use_layer_norm': True, 'box_size': 20.0}

Finished 0
Finished 200
Finished 400
Finished 600
Finished 800
cos value: 0.999333254594678
MAE 0.018202764408322854
std of MAE per sample: 0.0012427714472495637
RMSE: 0.025138951862883827
std of RMSE per sample: 0.004354936167142915
Relative MAE: 0.01688869452463752
Relative std: 0.0011530549352681203


In [ ]:
import sys, os
sys.path.append(os.path.join('../',os.path.dirname(os.path.abspath(''))))

from nn_module import WaterMDDynamicBoxNet
from train_utils import WaterDataRealLarge
from train_network_real_large import ParticleNetLightning
import torch
import numpy as np
from torch.utils.data import DataLoader
from types import SimpleNamespace

dataset = WaterDataRealLarge(dataset_path='./md_dataset/RPBE-data-processed.npz', mode='test')
dataloader = DataLoader(dataset, num_workers=2, batch_size=1, shuffle=False,
                          collate_fn=
                          lambda batches: {
                              'feat': torch.cat([torch.from_numpy(batch['feat']).float() for batch in batches], dim=0),
                              'pos': [batch['pos'] for batch in batches],
                              'forces': [batch['forces'] for batch in batches],
                              'box_size': [batch['box_size'] for batch in batches],
                          })

PATH = './model_ckpt_reallarge_noexpand/checkpoint_798.ckpt'
SCALER_CKPT = './model_ckpt_reallarge_noexpand/scaler_798.npz'
args = SimpleNamespace(use_layer_norm=True,
                       encoding_size=256,
                       hidden_dim=128,
                       edge_embedding_dim=256,
                      drop_edge=False,
                       conv_layer=5,
                      cutoff=9.5,
                      rotate_aug=False,
                       update_edge=False,
                       use_part=False,
                      data_dir='',
                      expand_edge=False,
                      loss='mae')
model = ParticleNetLightning(args).load_from_checkpoint(PATH, args=args)
model.load_training_stats(SCALER_CKPT)
model.cuda()
model.eval()
print('')
cos_vals = []
force = []
force_gt = []
mae = []
se = []
num = []
with torch.no_grad():
    count = 0
    for i_batch, batch in enumerate(dataloader):

        feat, pos, box_size = batch['feat'].float().cuda(), batch['pos'], batch['box_size']
        gt = torch.from_numpy(batch['forces'][0]).float().cuda()

        feat = feat.cuda()
        force_np = model.predict_forces(feat, pos[0], box_size[0])
        force_tsr = torch.from_numpy(force_np).cuda()
        cos_val = torch.sum(force_tsr * gt, dim=1) / (force_tsr.norm(dim=1)*gt.norm(dim=1))
        force_tsr = force_tsr.view(-1, 3)*(27/0.52917)  # hatree / bohr to ev/A
        gt = gt.view(-1, 3)*(27/0.52917)
        force += [force_tsr.cpu().numpy()]
        force_gt += [gt.cpu().numpy()]
        mae += [torch.sum(torch.mean(torch.abs(force_tsr - gt), dim=1)).cpu().numpy()]
        se += [torch.sum(torch.mean((force_tsr-gt)**2, dim=1)).cpu().numpy()]
        num += [pos[0].shape[0]]
        cos_vals += [cos_val]
        if count%200 == 0:
            print(f"Finished {i_batch}")
        count += 1
        
print(f'cos value: {torch.mean(torch.cat(cos_vals))}')
print(f'MAE {np.sum(mae) / np.sum(num)}')
print(f'std of MAE per sample: {np.std([mae[i]/num[i] for i in range(len(num))])}')
print(f'RMSE: {np.sqrt(np.sum(se)/ np.sum(num))}')
print(f'std of RMSE per sample: {np.std([np.sqrt(se[i]/num[i]) for i in range(len(num))])}')
print(f'Relative MAE: {(np.sum(mae) / np.sum(num))/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')
print(f'Relative std: {np.std([mae[i]/num[i] for i in range(len(num))])/ np.mean(np.linalg.norm(np.concatenate(force_gt), axis=-1))}')